In [1]:
import pandas as pd
import numpy as np
from HFC125_bmimAc_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.27
    init_kappa_1_2A = -0.11
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R125'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['bmimAc'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R125'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['bmimAc','bmimAc'].fix(0)
    m.fs.properties.PR_kappa_A['bmimAc', 'R125'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'bmimAc'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['bmimAc', 'bmimAc'].fix(0)
    m.fs.properties.PR_kappa_B['bmimAc', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'bmimAc'].fix(0)
    m.fs.properties.PR_kappa_C['bmimAc', 'bmimAc'].fix(0)
    m.fs.properties.PR_kappa_C['bmimAc', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'bmimAc'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['bmimAc'].unfix()
    m.fs.state_block.mole_frac_comp['R125'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data['x_R125']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'bmimAc'].fix(float(data['x_bmimAc']))
    m.fs.state_block.mole_frac_comp['R125'].fix(float(data['x_R125'])+eps)
    m.fs.state_block.mole_frac_comp['bmimAc'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['bmimAc', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_A['bmimAc', 'R125'].setub(5)

    m.fs.properties.PR_kappa_A['R125', 'bmimAc'].setlb(-5)
    m.fs.properties.PR_kappa_A['R125', 'bmimAc'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['bmimAc', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'bmimAc']"]


In [3]:
data = pd.read_csv('R125_bmimAc.csv')

print(data)

   temperature  pressure  x_R125  x_bmimAc
0       298.15     50000   0.060     0.940
1       298.15    100000   0.122     0.878
2       298.15    200000   0.240     0.760
3       298.15    400000   0.429     0.571
4       298.15    600000   0.536     0.464
5       298.15    800000   0.620     0.380
6       298.15   1000000   0.692     0.308


In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
    data_["x_bmimAc"] += data_["x_bmimAc"] * noise_x
    data_.loc[data_['x_bmimAc'] <= 0.0001, 'x_bmimAc'] = 0.0001
    data_.loc[data_['x_bmimAc'] >= 0.9999 , 'x_bmimAc'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
-0.16273329431020034
2022-03-06 21:10:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:10:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimAc,R125] = -0.2877362347829211
fs.properties.PR_kappa_A[R125,bmimAc] = -0.12937478315065046
1
0.05822

2022-03-06 21:11:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimAc,R125] = -0.2625810478176559
fs.properties.PR_kappa_A[R125,bmimAc] = -0.10657723888266488
8
-0.1554765778302308
2022-03-06 21:11:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:11:23 [INFO] idaes.init.fs.state_block: Property 

2022-03-06 21:12:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:12:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:12:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:12:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:12:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimAc,R125] = -0.26891306184595437
fs.properties.PR_kappa_A[R125,bmimAc] = -0.10655775578990315
15
-0.1463936353543162
2022-03-06 21:12:28 [INFO] idaes.init.fs.state_block: Propert

2022-03-06 21:13:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimAc,R125] = -0.2612533271276847
fs.properties.PR_kappa_A[R125,bmimAc] = -0.10655555256879487
22
-0.08107441727235401
2022-03-06 21:13:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:13:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:13:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:13:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:13:40 [INFO] idaes.init.fs.state_block: Propert

2022-03-06 21:14:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:14:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:14:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:14:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:14:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:14:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:14:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimAc,R125] = -0.28723114627213725
fs.properties.PR_kappa_A[R125,bmimAc] = -0.12585630701802553
30
0.038008836824081876
2022-03

...

2022-03-06 21:20:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:20:46 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[bmimAc,R125] = -0.2626593601001632
fs.properties.PR_kappa_A[R125,bmimAc] = -0.10657690374732262
67
0.013416126417141419
2022-03-06 21:20:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:20:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:20:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:20:52 [INFO] idaes.init.fs.state_block: Propert

2022-03-06 21:21:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:21:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimAc,R125] = -0.28908245849026676
fs.properties.PR_kappa_A[R125,bmimAc] = -0.1398949065284401
74
-0.12357862843450121
2022-03-06 21:21:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:21:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:21:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:22:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:22:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-

fs.properties.PR_kappa_A[bmimAc,R125] = -0.26860705594510964
fs.properties.PR_kappa_A[R125,bmimAc] = -0.10656132181041394
81
0.04419366623640958
2022-03-06 21:23:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:23:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    mode

2022-03-06 21:24:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[bmimAc,R125] = -0.28646980776898157
fs.properties.PR_kappa_A[R125,bmimAc] = -0.12096831548981941
89
0.20285052805359075
2022-03-06 21:24:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:24:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-

fs.properties.PR_kappa_A[bmimAc,R125] = -0.28693371273321006
fs.properties.PR_kappa_A[R125,bmimAc] = -0.12388942859738489
96
-0.20375944307420157
2022-03-06 21:25:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 21:25:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.prop

In [5]:
print(results_params)

[[-0.28773623 -0.12937478]
 [-0.2586389  -0.10655343]
 [-0.28767289 -0.1289209 ]
 [-0.27389207 -0.10650796]
 [-0.28510731 -0.11329812]
 [-0.25327101 -0.10644605]
 [-0.28751571 -0.12781039]
 [-0.26258105 -0.10657724]
 [-0.28762088 -0.12855091]
 [-0.26632899 -0.10657949]
 [-0.2621867  -0.10657872]
 [-0.2794914  -0.10645571]
 [-0.26661875 -0.10657762]
 [-0.25437995 -0.10646636]
 [-0.26891306 -0.10655776]
 [-0.28747582 -0.12753209]
 [-0.28707745 -0.1248305 ]
 [-0.26379072 -0.10657195]
 [-0.28877289 -0.13733047]
 [-0.25344274 -0.106439  ]
 [-0.28718756 -0.12556332]
 [-0.26125333 -0.10655555]
 [-0.28640584 -0.12057889]
 [-0.28517911 -0.11367235]
 [-0.25302603 -0.10651955]
 [-0.2869814  -0.1241997 ]
 [-0.28626904 -0.11975659]
 [-0.28611062 -0.10639096]
 [-0.25970066 -0.10654694]
 [-0.28723115 -0.12585631]
 [-0.25623268 -0.10652399]
 [-0.27459349 -0.10650325]
 [-0.28723741 -0.12589856]
 [-0.26230943 -0.10657849]
 [-0.26531782 -0.10656516]
 [-0.28560351 -0.11594818]
 [-0.28685347 -0.1233716 ]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R125_bmimAc_10_2.csv')